# Problem Set 3 problems

## Question 1: Double pendulum

### Learning objectives
In this question you will:

- study the double pendulum theoretically using Lagrangian mechanics
- numerically solve differential equations that can't be solved analytically
- assess the effectiveness of numerics by comparing theoretical expectations to numerical results
- understand what people mean by "chaos" by analysing a chaotic system


The double pendulum is pictured below:

<div style="width: 500px;margin: auto" align="center">
    <img src="double_pendulum.PNG">
</div>

### 1a. 

Show that the Lagrangian of the system is given by $$L=\tfrac{1}{2}(m_1+m_2)L_1^2\dot\phi_1^2+\tfrac{1}{2}m_2L_2^2\dot\phi_2^2+m_2L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_2-\phi_1)-(m_1+m_2)gL_1(1-\cos\phi_1)-m_2gL_2(1-\cos\phi_2).$$

Write your answer here

### 1b. 

Find the equations of motion.

Write your answer here

### 1c. 

Under the small angle approximation $\phi_1,\phi_2\ll1$ (keeping only first-order terms), find the normal modes of oscillation, which is pictured below for a simple choice of parameters.

<div style="width: 500px;margin: auto" align="center">
    <img src="normal_modes.gif">
    The normal modes for $m_1=m_1$, $L_1=L_2$, and time measured in natural units. (Angles are exaggerated; this is only valid in the small-angle regime.)
</div>

Write your answer here

### 1d. 

Now use your theoretical solution for small angles to fill in the following Python function that, given the initial state of the double pendulum, returns the states at some specified times. Use the convention $(\phi_1,\phi_2,\dot\phi_1,\dot\phi_2)^T$ for the state. (Hint: transform to the basis of normal modes and then transform back. The usual mapping of $\mathbb{R}^{2n}$ to $\mathbb{C}^n$ might be useful here.)

In [ ]:
import numpy as np

def small_angle_soln(initial_state, tmax, N, m1=1, m2=1, L1=1, L2=1, g=1):
    """
    Returns: tuple (times, states) where:
    times is an array of shape (N,) evenly sampled from 0 to tmax,
    states is array of shape (4,N) containing corresponding states in 
        the format (phi_1, phi_2, d/dt phi_1, d/dt phi_2)
    """
    times = np.linspace(0,tmax,N)
    
    phi1 = 
    phi2 = 
    dotphi1 = 
    dotphi2 = 
    
    return times, np.array([phi1, phi2, dotphi1, dotphi2])

### 1e. 

The following cells contains a visualisation function that animates the double pendulum given a list of times and state vectors. You might find it useful.

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display,HTML

plt.rcParams["animation.embed_limit"] = 200 
#sets the max animation size to 200MB so that you can make long animations if you want

def animate(times, states, c="k", m1=1, m2=1, L1=1, L2=1, g=1, labels=None):
    """
    Returns animation of pendula (or pendulum). 
    times must be array of shape (N,).
    states must be array of shape (4,N), or list-like of n such arrays. (n is # of pendula)
        states must follow the usual convention.
    c must be list-like of length n. c[i] will be the colour of pendulum i.
    m1...g are problem parameters (g has no effect here)
    if labels are provided, must be list-like of length n. labels[i] will be label of pendulum i.
    Returns nothing, but displays the animation.
    """
    #change matplotlib backend so plots aren't displayed (only animation is)
    %matplotlib agg
    n = len(c) #number of pendula
    N = len(times) #number of frames
    xs, ys = [], [] #positions of pendula (store them to update them during animation)
    strings = [] #line objects depicting strings (store them to update them during animation)
    pendula = [] #(tuples of) circle objects depiciting masses (store them to update them during animation)
    f = plt.figure(figsize=(6,6))
    labels = [None]*n if labels is None else labels
    for i in range(n):
        state = states[i] if n>1 else angles #get (4,N) state vector array
        x,y = np.zeros((N,3)), np.zeros((N,3)) #positions of string ends (which contains positions of pendula)
        y[:,1] = -L1*np.cos(state[0])
        x[:,1] = L1*np.sin(state[0])
        y[:,2] = y[:,1] - L2*np.cos(state[1])
        x[:,2] = x[:,1] + L2*np.sin(state[1])
        xs.append(x) #store positions
        ys.append(y)
        args = {"color": c[i], "alpha": sqrt(1/n)} #plotting args
        strings.append(plt.plot(x[0], y[0], label=labels[i], **args)[0]) #plot string at initial posn, store line object
        pend = plt.Circle((x[0,1],y[0,1]), m1**(1/3)/10, **args), plt.Circle((x[0,2],y[0,2]), m2**(1/3)/10, **args)
        [plt.gca().add_artist(p) for p in pend] #plot masses at initial posn
        pendula.append(pend) #store circle
    tit = plt.title(f"$t=0$") #store title object to update time
    padding = m2**(1/3)/10+0.1
    plt.xlim(-L1-L2-padding,L1+L2+padding) #set bounds so pundulum is always displayed
    plt.ylim(-L1-L2-padding,L1+L2+padding)
    to_ret = [tit]+strings #list of things to be updated at each frame (for blitting)
    for p in pendula:
        to_ret += list(p)
    plt.xticks([]) #don't display the scale of axes
    plt.yticks([])
    plt.gca().set_aspect("equal") #set aspect ratio to 1
    if labels[0] is not None: #add legend if wanted
        plt.legend(frameon=False, loc="upper left")
    
    def update(j):
        tit.set_text(f"$t={times[j]:.2f}$") #update time in title
        for i in range(n):
            strings[i].set_data(xs[i][j],ys[i][j]) #update string ends
            pendula[i][0].set_center((xs[i][j,1],ys[i][j,1])) #update first mass
            pendula[i][1].set_center((xs[i][j,2],ys[i][j,2])) #update second mass
        return to_ret #return changed elements for blitting
    
    anim = FuncAnimation(f, update, range(N), interval=1e3/24, blit=True) #24 fps
    display(HTML(anim.to_jshtml())) #output animation
    %matplotlib inline

As a test of your `small_angle_soln()`, and to show you how to use the `animate()` function, fill in the appropriate initial states to recover the normal modes for $m_1=m_2$ and $L_1=L_2$, and compare your animation to the one embedded above.

In [ ]:
initial_state_1 = #answer here
initial_state_2 = #answer here

tmax = 20
N = 200

times, normal_mode_1 = small_angle_soln(initial_state_1, tmax, N)
times, normal_mode_2 = small_angle_soln(initial_state_2, tmax, N)

animate(times, (normal_mode_1, normal_mode_2), "rb")

### 1f. 

The following cells contain a visualisation function that plots a sampled trajectory in phase space.

In [4]:
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib import cm as cmaps

def plot_phase_space(times, states, size=1, cmap=cmaps.rainbow):
    """
    times must be array of shape (N,) and states must be array of shape (4,N) following usual conventions.
    size is the size of marker (you might want to reduce it if N is large).
    cmap must be a matplotlib colourmap.
    Returns nothing, plots the sampled trajectory in 4-d phase space as 6 2-d plots. 
    """
    f,ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,12), gridspec_kw={"hspace": 0.02, "wspace": 0.02})
    labels = [r"$\phi_1$", r"$\phi_2$", r"$\dot\phi_1$", r"$\dot\phi_2$"]
    args = {"c": times, "s": size, "cmap": cmap}
    for i in range(3):
        ax[i,0].set_ylabel(labels[i+1])
        ax[-1,i].set_xlabel(labels[i])
        for j in range(i+1):
            ax[i,j].scatter(states[j], states[i+1], **args)
        for j in range(i+1,3):
            ax[i,j].axis("off")
    cbax =  f.add_axes([0.58,0.87,0.3,.01])
    ColorbarBase(cbax, cmap=cmap, norm=Normalize(times[0], times[-1]), orientation="horizontal", label="$t$")

Run the following cell to visualise the solutions you animated above in phase space. Is this what you expect?

In [ ]:
plot_phase_space(times, normal_mode_1)

In [ ]:
plot_phase_space(times, normal_mode_2)

Now play around with the system. Plot phase space trajectories for some other initial states that are not normal modes. Use large values for $t_\max$ and $N$. What does the phase space trajectory look like? Is it periodic, or dense in some allowed phase space? 

An _ergodic_ system is a system in which from any starting state, one reaches arbitrarily close to any given state at some time. Does this system (linearised under the small-angle approximation) display ergodicity?

In [ ]:
plot_phase_space(*small_angle_soln([0,1,0,0],500,10000))

### 1g. 

Rewrite the (full) equations of motion as a (non-linear) four-dimensional first-order ODE. Use the same convention for the state, $(\phi_1,\phi_2,\dot\phi_1,\dot\phi_2)^T$.

Write your answer here

### 1h. 

Implement your answer above in the following function: 

In [ ]:
def derivative(state, m1=1, m2=1, L1=1, L2=1, g=1):
    """Given state of shape (4,) as well as the problem parameters, 
    returns an array of shape (4,) representing the time-derivative of the state"""
    der = 
    #fill in your answer here
    return der

### 1i. 

Solve the EoMs analytically.

Lol jk, let's use some differential equation solvers to solve the full, non-linear EoMs which we can't solve analytically. The functions given below are implementations of the Euler, Symplectic Euler, and a 5<sup>th</sup> order Runge-Kutta method (these methods are introduced in the [introductory Python notebooks](https://github.com/avirukt/intro_python/blob/master/Intro%20to%20numerics.ipynb)) and follow the same conventions (and have the same call signature) as `small_angle_soln()`. They will solve the ODE defined by your implementation of `derivative()`, which, if you did everything right, are the EoMs of the double pendulum.

In [10]:
from scipy.integrate import solve_ivp

def euler(initial_state, tmax, N, **extra_args):
    times = np.linspace(0,tmax,N)
    dt = times[1]-times[0]
    states = np.zeros((N,4))
    states[0] = initial_state
    for i in range(N-1):
        states[i+1] = states[i] + derivative(states[i], **extra_args)*dt
    return times, states.T
    
def symplectic_euler(initial_state, tmax, N, **extra_args):
    times = np.linspace(0,tmax,N)
    dt = times[1]-times[0]
    states = np.zeros((N,4))
    states[0] = initial_state
    for i in range(N-1):
        states[i+1,2:] = states[i,2:] + derivative(states[i], **extra_args)[2:]*dt
        states[i+1,:2] = states[i,:2] + states[i+1,2:]*dt
    return times, states.T

def runge_kutta(initial_state, tmax, N, **extra_args):
    times = np.linspace(0,tmax,N)
    fn = lambda t,y: derivative(y, **extra_args)
    soln = solve_ivp(fn, (0, tmax), initial_state, max_step=tmax/(N+1), dense_output=True).sol
    return times, soln(times)

Here is a function that will plot phase-space plots (and, optionally, an animation) of trajectories sampled using each of the three methods (and, optionally, the theoretical small-angle solution) for comparison. There is also a helper function to bring angles within the interval $[-\pi,\pi)$, which might be useful later.

In [11]:
def bound_angles(states):
    """Returns copy of states, where angles are in the interval [-pi,pi)"""
    states = states.copy()
    states[:2] = (states[:2]+np.pi)%(2*np.pi)-np.pi
    return states

def compare_methods(*args, theo=False, anim=True, energies=False, **extra_args):
    """
    First three args must be initial_state, tmax and N.
    theo is boolean for whether or not to include theoretical small-angle solution in comparison.
    anim is boolean for whether or not to animate the trajectories (takes a while, so don't use for large N).
    energies is boolean for whether or not to plot the energies as well. (requires function "energy" to be defined)
    extra_args are problem parameters.
    Returns nothing, solves ODE using the 3-4 methods, and plots phase-space trajectories (and animations).
    """
    labels = ["Euler", "symplectic Euler", "5th-order Runge-Kutta"]
    methods = [euler, symplectic_euler, runge_kutta]
    colours = "rgb"
    if theo:
        labels += ["theoretical (small-angle)"]
        methods += [small_angle_soln]
        colours ="rgbk"
    ys=[]
    for meth in methods:
        t,a = meth(*args, **extra_args)
        ys.append(a)
    if anim:
        extra_args["labels"] = labels
        animate(t, ys, colours, **extra_args)
    for i in range(len(labels)):
        plot_phase_space(t,bound_angles(ys[i]))
        plt.suptitle(labels[i])
    if energies:
        plt.figure()
        for i in range(len(labels)):
            plt.plot(t,energy(ys[i],**extra_args),label=labels[i])
        plt.axhline(energy(args[0],**extra_args),color="k",ls="--",label="true value")
        plt.legend(frameon=False)
        plt.xlabel("time")
        plt.ylabel("energy")
        plt.yscale("log")

For a small angle, compare the three methods to the theoretical solution. Which methods are able to capture the continuous behaviour, and which aren't? Simply running the following cell will give you lots of information.

In [ ]:
%time compare_methods([0.01,0,0,0], 60, 500, theo=True)

Write your answer here

### 1j. 

Compare the three methods outside of the small-angle regime. For how long do they display qualitatively similar behaviour? Which method, if any, can we trust?

In [ ]:
#Write your answer here

### 1k. 

A good test that the differential equation solvers aren't accumulating errors is to check if constants of the motion are conserved (the ODE solvers don't know about conserved quantities, so they don't enforce them; if they did they would be more efficient). Here, the only (known) conserved quantity is energy. Why is that?

Write your answer here

### 1l. 

Implement the following function to calculate the energies of state vectors. After implementing this function, you can pass `energies=True` to `compare_methods()` to also plot the energies of the trajectories, to check if it is constant.

In [ ]:
def energy(states, m1=1, m2=1, L1=1, L2=1, g=1):
    """states is array of shape (4,N). Returns energies in array of shape (N,)."""
    return #fill me in

### 1m. 

Now, integrating to large times (you might want to turn off animation), compare the three methods outside of the small-angle regime, also comparing the energies. How much can we trust the best method, and upto what sort of time scale?

In [ ]:
#Write your answer here

### 1n. 

You might have heard that the double pendulum is chaotic. This means that it is extremely sensitive to initial conditions. To demonstrate this, make an animation of a few pendula that start off very close together but then diverge. Verify that the solver you use isn't giving you terribly unreasonable results.

In [ ]:
#Write your answer here

### 1o. 

Chaotic systems aren't just unstable to perturbations: they are __extremely__ unstable to perturbations. Perturbations must grow exponentially for a system to be chaotic. Choose an appropriate initial state and find its trajectory. Then perturb it slightly in a few different ways, and track the differences in the trajectories. How quickly do perturbations grow? Is the double pendulum chaotic inside the small-angle regime (remember that most trajectories are dense)? How about outside it?

In [ ]:
#Write your answer here

### 1p. 

Suppose an experimenter initialises a pendulum to some state. Now, in the real world, we only have access to a finite level of accuracy. Assuming that the error in the initialisation/measurement of each parameter is independent and Gaussian, argue that the pendulum is not described by a single state, but by a Gaussian distribution over states. What happens to this distribution over time?

Write your answer here